## **First attempt at CNN MIT-BIH**

In [1]:
# Load packages and Data

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

mitbih_test_df = pd.read_csv("mitbih_test.csv")
mitbih_train_df = pd.read_csv("mitbih_train.csv")

In [2]:
# Renaming the columns
new_column_names = range(1, len(mitbih_train_df.columns) + 1)
mitbih_train_df.columns = new_column_names

new_column_names = range(1, len(mitbih_test_df.columns) + 1)
mitbih_test_df.columns = new_column_names

# Transforming the Target Variable to Integer
mitbih_train_df[188]=mitbih_train_df[188].astype(int)

# Transforming the Target Variable to Integer
mitbih_test_df[188]=mitbih_test_df[188].astype(int)

print(mitbih_train_df[188].value_counts())

print(mitbih_test_df.shape)
print(mitbih_train_df.shape)

#Shuffle the rows of the DFs
mitbih_test_df = mitbih_test_df.sample(frac=1).reset_index(drop=True)
mitbih_train_df = mitbih_train_df.sample(frac=1).reset_index(drop=True)

mitbih_train_df.head()

188
0    72470
4     6431
2     5788
1     2223
3      641
Name: count, dtype: int64
(21891, 188)
(87553, 188)


,1,2,3,4,5,6,7,8,9,10,...,179,180,181,182,183,184,185,186,187,188
0,1.000000,0.956811,0.594684,0.076412,0.046512,0.172757,0.126246,0.086379,0.076412,0.063123,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1.000000,0.543307,0.335958,0.173228,0.115486,0.044619,0.028871,0.020997,0.010499,0.010499,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.967302,0.844687,0.267030,0.000000,0.117166,0.247956,0.291553,0.288828,0.283379,0.280654,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.875670,0.823151,0.757771,0.694534,0.604502,0.497320,0.375134,0.233655,0.128617,0.086817,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
4,0.927869,0.960656,0.698361,0.311475,0.000000,0.065574,0.124590,0.180328,0.209836,0.206557,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [4]:
# Load ML-Packages

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler,  ClusterCentroids
from imblearn.metrics import classification_report_imbalanced, geometric_mean_score
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate

### Basic Model: 

In [4]:
X_train = mitbih_train_df.iloc[:, :-1]

y_train = mitbih_train_df.iloc[:, -1]

X_test = mitbih_test_df.iloc[:, :-1]
y_test = mitbih_test_df.iloc[:, -1]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(87553, 187) (87553,)
(21891, 187) (21891,)


In [5]:
from tensorflow.keras.utils import to_categorical

# One-Hot-Encoding der Zielvariablen für Mehrklassenklassifikation
y_train = to_categorical(y_train, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)

In [11]:
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

# Modell Input
inputs = Input(shape=(187, 1), name="Input")

# 1D-CNN Layer mit Batch Normalization
first_layer = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
second_layer = BatchNormalization()(first_layer)
third_layer = MaxPooling1D(pool_size=2)(second_layer)

# Zweite CNN-Schicht hinzufügen
fourth_layer = Conv1D(filters=128, kernel_size=3, activation='relu')(third_layer)
fifth_layer = BatchNormalization()(fourth_layer)
sixth_layer = MaxPooling1D(pool_size=2)(fifth_layer)

# LSTM Layer
lstm_layer = LSTM(100, return_sequences=False)(sixth_layer)

# Dense Layers
dense_layer = Dense(50, activation='relu')(lstm_layer)
dropout_layer = Dropout(0.5)(dense_layer)

# Output Layer für mehrklassige Klassifikation
output_layer = Dense(5, activation='softmax')(dropout_layer)

# Modell erstellen
model = Model(inputs=inputs, outputs=output_layer)

# Modell kompilieren
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modell-Übersicht anzeigen
model.summary()

# Early Stopping Callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Modell trainieren
training_history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, callbacks=[early_stopping])


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input (InputLayer)              │ (None, 187, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 185, 64)        │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 185, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 92, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 90, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 90, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 45, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100)            │        91,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           255 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 122,633 (479.04 KB)

 Trainable params: 122,249 (477.54 KB)

 Non-trainable params: 384 (1.50 KB)

Epoch 1/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 35s 15ms/step - accuracy: 0.8723 - loss: 0.4875 - val_accuracy: 0.9475 - val_loss: 0.1999
Epoch 2/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 33s 15ms/step - accuracy: 0.9525 - loss: 0.1809 - val_accuracy: 0.9617 - val_loss: 0.1398
Epoch 3/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 33s 15ms/step - accuracy: 0.9632 - loss: 0.1412 - val_accuracy: 0.9643 - val_loss: 0.1314
Epoch 4/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 33s 15ms/step - accuracy: 0.9698 - loss: 0.1175 - val_accuracy: 0.9711 - val_loss: 0.1073
Epoch 5/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 34s 15ms/step - accuracy: 0.9725 - loss: 0.1048 - val_accuracy: 0.9680 - val_loss: 0.1282
Epoch 6/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 41s 19ms/step - accuracy: 0.9742 - loss: 0.0984 - val_accuracy: 0.9648 - val_loss: 0.1266
Epoch 7/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 36s 17ms/step - accuracy: 0.9766 - loss: 0.0888 - val_accuracy: 0.9714 - val_loss: 0.1060
Epoch 8/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 36s 16ms/step - accuracy: 0.9779 -

In [12]:
# Prediction: 

from sklearn.metrics import classification_report
import numpy as np

y_pred = model.predict(X_test)

y_pred_classes = np.argmax(y_pred, axis=1)

y_true_classes = np.argmax(y_test, axis=1)

report = classification_report(y_true_classes, y_pred_classes)

print(report)

685/685 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     18117
           1       0.84      0.69      0.76       556
           2       0.98      0.92      0.95      1448
           3       0.74      0.81      0.78       162
           4       1.00      0.98      0.99      1608

    accuracy                           0.98     21891
   macro avg       0.91      0.88      0.89     21891
weighted avg       0.98      0.98      0.98     21891



### With Oversampling with SMOTE:

In [19]:
X_train = mitbih_train_df.iloc[:, :-1]

y_train = mitbih_train_df.iloc[:, -1]

X_test = mitbih_test_df.iloc[:, :-1]
y_test = mitbih_test_df.iloc[:, -1]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(87553, 187) (87553,)
(21891, 187) (21891,)


In [22]:
smo=SMOTE()
X_sm,y_sm = smo.fit_resample(X_train, y_train)
print("Klassen Anzahl y-train: ", dict(pd.Series(y_train).value_counts()))
print("Klassen Anzahl Oversampled (SMO): ", dict(pd.Series(y_sm).value_counts()))
print("Shape x_sm: ", X_sm.shape)
print("Shape y_sm: ", y_sm.shape)

Klassen Anzahl y-train:  {0: 72470, 4: 6431, 2: 5788, 1: 2223, 3: 641}
Klassen Anzahl Oversampled (SMO):  {0: 72470, 4: 72470, 2: 72470, 1: 72470, 3: 72470}
Shape x_sm:  (362350, 187)
Shape y_sm:  (362350,)


In [23]:
from tensorflow.keras.utils import to_categorical

# One-Hot-Encoding der Zielvariablen für Mehrklassenklassifikation
y_sm = to_categorical(y_sm, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)

In [24]:
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

# Modell Input
inputs = Input(shape=(187, 1), name="Input")

# 1D-CNN Layer mit Batch Normalization
first_layer = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
second_layer = BatchNormalization()(first_layer)
third_layer = MaxPooling1D(pool_size=2)(second_layer)

# Zweite CNN-Schicht hinzufügen
fourth_layer = Conv1D(filters=128, kernel_size=3, activation='relu')(third_layer)
fifth_layer = BatchNormalization()(fourth_layer)
sixth_layer = MaxPooling1D(pool_size=2)(fifth_layer)

# LSTM Layer
lstm_layer = LSTM(100, return_sequences=False)(sixth_layer)

# Dense Layers
dense_layer = Dense(50, activation='relu')(lstm_layer)
dropout_layer = Dropout(0.5)(dense_layer)

# Output Layer für mehrklassige Klassifikation
output_layer = Dense(5, activation='softmax')(dropout_layer)

# Modell erstellen
model = Model(inputs=inputs, outputs=output_layer)

# Modell kompilieren
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modell-Übersicht anzeigen
model.summary()

# Early Stopping Callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Modell trainieren
training_history = model.fit(X_sm, y_sm, validation_split=0.2, epochs=50, batch_size=32, callbacks=[early_stopping])

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input (InputLayer)              │ (None, 187, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 185, 64)        │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 185, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 92, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 90, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 90, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 45, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 100)            │        91,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 5)              │           255 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 122,633 (479.04 KB)

 Trainable params: 122,249 (477.54 KB)

 Non-trainable params: 384 (1.50 KB)

Epoch 1/50
9059/9059 ━━━━━━━━━━━━━━━━━━━━ 254s 28ms/step - accuracy: 0.8234 - loss: 0.4888 - val_accuracy: 0.9364 - val_loss: 0.2416
Epoch 2/50
9059/9059 ━━━━━━━━━━━━━━━━━━━━ 386s 43ms/step - accuracy: 0.9604 - loss: 0.1299 - val_accuracy: 0.9624 - val_loss: 0.1382
Epoch 3/50
9059/9059 ━━━━━━━━━━━━━━━━━━━━ 360s 40ms/step - accuracy: 0.9748 - loss: 0.0832 - val_accuracy: 0.9623 - val_loss: 0.1658
Epoch 4/50
9059/9059 ━━━━━━━━━━━━━━━━━━━━ 208s 23ms/step - accuracy: 0.9801 - loss: 0.0655 - val_accuracy: 0.9635 - val_loss: 0.1787
Epoch 5/50
9059/9059 ━━━━━━━━━━━━━━━━━━━━ 219s 24ms/step - accuracy: 0.9839 - loss: 0.0533 - val_accuracy: 0.8754 - val_loss: 0.5235
Epoch 6/50
9059/9059 ━━━━━━━━━━━━━━━━━━━━ 198s 22ms/step - accuracy: 0.9866 - loss: 0.0451 - val_accuracy: 0.9482 - val_loss: 0.4476
Epoch 7/50
9059/9059 ━━━━━━━━━━━━━━━━━━━━ 181s 20ms/step - accuracy: 0.9875 - loss: 0.0413 - val_accuracy: 0.9688 - val_loss: 0.1528
Epoch 8/50
9059/9059 ━━━━━━━━━━━━━━━━━━━━ 241s 27ms/step - accuracy: 

In [25]:
# Prediction: 

from sklearn.metrics import classification_report
import numpy as np

y_pred = model.predict(X_test)

y_pred_classes = np.argmax(y_pred, axis=1)

y_true_classes = np.argmax(y_test, axis=1)

report = classification_report(y_true_classes, y_pred_classes)

print(report)

685/685 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step
              precision    recall  f1-score   support

           0       0.99      0.98      0.99     18117
           1       0.71      0.83      0.76       556
           2       0.94      0.96      0.94      1448
           3       0.63      0.86      0.73       162
           4       0.97      0.98      0.98      1608

    accuracy                           0.97     21891
   macro avg       0.85      0.92      0.88     21891
weighted avg       0.98      0.97      0.98     21891



### With Class weight balance: 

In [37]:
X_train = mitbih_train_df.iloc[:, :-1]

y_train = mitbih_train_df.iloc[:, -1]

X_test = mitbih_test_df.iloc[:, :-1]
y_test = mitbih_test_df.iloc[:, -1]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(87553, 187) (87553,)
(21891, 187) (21891,)


In [38]:
# Berechne die Class Weights basierend auf der Klassenverteilung
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(y_train),
                                                  y=y_train)

# Konvertiere die Class Weights in ein Dictionary
class_weight_dict = dict(enumerate(class_weights))

print("Class Weights:", class_weight_dict)

Class Weights: {0: 0.2416255002069822, 1: 7.877013045434098, 2: 3.025328265376641, 3: 27.317628705148206, 4: 2.7228424817291246}


In [39]:
from tensorflow.keras.utils import to_categorical

# One-Hot-Encoding der Zielvariablen für Mehrklassenklassifikation
y_train = to_categorical(y_train, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)

In [41]:
# With Batch normalization: 

from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import class_weight
import numpy as np

# Modell Input
inputs = Input(shape=(187, 1), name="Input")

# 1D-CNN Layer mit Batch Normalization
first_layer = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
second_layer = BatchNormalization()(first_layer)
third_layer = MaxPooling1D(pool_size=2)(second_layer)

# Zweite CNN-Schicht hinzufügen
fourth_layer = Conv1D(filters=128, kernel_size=3, activation='relu')(third_layer)
fifth_layer = BatchNormalization()(fourth_layer)
sixth_layer = MaxPooling1D(pool_size=2)(fifth_layer)

# LSTM Layer
lstm_layer = LSTM(100, return_sequences=False)(sixth_layer)

# Dense Layers
dense_layer = Dense(50, activation='relu')(lstm_layer)
dropout_layer = Dropout(0.5)(dense_layer)

# Output Layer für mehrklassige Klassifikation
output_layer = Dense(5, activation='softmax')(dropout_layer)

# Modell erstellen
model = Model(inputs=inputs, outputs=output_layer)

# Modell kompilieren
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modell-Übersicht anzeigen
model.summary()

# Early Stopping Callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Modell trainieren mit Class Weights
training_history = model.fit(X_train, y_train, 
                             validation_split=0.2, 
                             epochs=50, 
                             batch_size=32, 
                             class_weight=class_weight_dict,
                             callbacks=[early_stopping])

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input (InputLayer)              │ (None, 187, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_14 (Conv1D)              │ (None, 185, 64)        │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 185, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_14 (MaxPooling1D) │ (None, 92, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_15 (Conv1D)              │ (None, 90, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 90, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_15 (MaxPooling1D) │ (None, 45, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 100)            │        91,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 5)              │           255 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 122,633 (479.04 KB)

 Trainable params: 122,249 (477.54 KB)

 Non-trainable params: 384 (1.50 KB)

Epoch 1/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 45s 20ms/step - accuracy: 0.3178 - loss: 1.3553 - val_accuracy: 0.7526 - val_loss: 0.7850
Epoch 2/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 41s 19ms/step - accuracy: 0.6710 - loss: 0.6436 - val_accuracy: 0.8652 - val_loss: 0.5349
Epoch 3/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 41s 19ms/step - accuracy: 0.7953 - loss: 0.5356 - val_accuracy: 0.8576 - val_loss: 0.5407
Epoch 4/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 43s 20ms/step - accuracy: 0.8312 - loss: 0.4235 - val_accuracy: 0.7520 - val_loss: 0.7011
Epoch 5/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 45s 21ms/step - accuracy: 0.8345 - loss: 0.3835 - val_accuracy: 0.8887 - val_loss: 0.4454
Epoch 6/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 101s 46ms/step - accuracy: 0.8678 - loss: 0.3529 - val_accuracy: 0.8694 - val_loss: 0.4847
Epoch 7/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 100s 46ms/step - accuracy: 0.8691 - loss: 0.3634 - val_accuracy: 0.8750 - val_loss: 0.3875
Epoch 8/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 101s 46ms/step - accuracy: 0.882

In [42]:
# Prediction: 

from sklearn.metrics import classification_report
import numpy as np

y_pred = model.predict(X_test)

y_pred_classes = np.argmax(y_pred, axis=1)

y_true_classes = np.argmax(y_test, axis=1)

report = classification_report(y_true_classes, y_pred_classes)

print(report)

685/685 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step
              precision    recall  f1-score   support

           0       0.99      0.93      0.96     18117
           1       0.39      0.82      0.53       556
           2       0.87      0.95      0.90      1448
           3       0.26      0.90      0.41       162
           4       0.95      0.99      0.97      1608

    accuracy                           0.93     21891
   macro avg       0.69      0.92      0.75     21891
weighted avg       0.96      0.93      0.94     21891



### Without Batch Normalization (With optimized Params): 

In [43]:
X_train = mitbih_train_df.iloc[:, :-1]

y_train = mitbih_train_df.iloc[:, -1]

X_test = mitbih_test_df.iloc[:, :-1]
y_test = mitbih_test_df.iloc[:, -1]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(87553, 187) (87553,)
(21891, 187) (21891,)


In [44]:
from tensorflow.keras.utils import to_categorical

# One-Hot-Encoding der Zielvariablen für Mehrklassenklassifikation
y_train = to_categorical(y_train, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)

In [45]:
# Without batch Normalization: 

from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import class_weight
import numpy as np

# Modell Input
inputs = Input(shape=(187, 1), name="Input")

# 1D-CNN Layer ohne Batch Normalization
first_layer = Conv1D(filters=64, kernel_size=5, activation='relu')(inputs)
third_layer = MaxPooling1D(pool_size=2)(first_layer)

# Zweite CNN-Schicht hinzufügen
fourth_layer = Conv1D(filters=128, kernel_size=7, activation='relu')(third_layer)
sixth_layer = MaxPooling1D(pool_size=2)(fourth_layer)

# LSTM Layer
lstm_layer = LSTM(100, return_sequences=False)(sixth_layer)

# Dense Layers
dense_layer = Dense(100, activation='relu')(lstm_layer)
dropout_layer = Dropout(0.8)(dense_layer)

# Output Layer für mehrklassige Klassifikation
output_layer = Dense(5, activation='softmax')(dropout_layer)

# Modell erstellen
model = Model(inputs=inputs, outputs=output_layer)

# Modell kompilieren
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modell-Übersicht anzeigen
model.summary()

# Early Stopping Callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)


# Modell trainieren mit Class Weights
training_history = model.fit(X_train, y_train, 
                             validation_split=0.2, 
                             epochs=50, 
                             batch_size=16, 
                             callbacks=[early_stopping])

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input (InputLayer)              │ (None, 187, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_16 (Conv1D)              │ (None, 183, 64)        │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_16 (MaxPooling1D) │ (None, 91, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_17 (Conv1D)              │ (None, 85, 128)        │        57,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_17 (MaxPooling1D) │ (None, 42, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 100)            │        91,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 5)              │           505 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 160,061 (625.24 KB)

 Trainable params: 160,061 (625.24 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 45s 10ms/step - accuracy: 0.8441 - loss: 0.6254 - val_accuracy: 0.9290 - val_loss: 0.2248
Epoch 2/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 44s 10ms/step - accuracy: 0.9329 - loss: 0.2437 - val_accuracy: 0.9539 - val_loss: 0.1584
Epoch 3/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 43s 10ms/step - accuracy: 0.9544 - loss: 0.1702 - val_accuracy: 0.9555 - val_loss: 0.1571
Epoch 4/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 44s 10ms/step - accuracy: 0.9623 - loss: 0.1422 - val_accuracy: 0.9692 - val_loss: 0.1144
Epoch 5/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 44s 10ms/step - accuracy: 0.9664 - loss: 0.1254 - val_accuracy: 0.9660 - val_loss: 0.1157
Epoch 6/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 45s 10ms/step - accuracy: 0.9698 - loss: 0.1152 - val_accuracy: 0.9749 - val_loss: 0.0945
Epoch 7/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 48s 11ms/step - accuracy: 0.9733 - loss: 0.1012 - val_accuracy: 0.9722 - val_loss: 0.1032
Epoch 8/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 1606s 367ms/step - accuracy: 0.973

In [46]:
# Prediction: 

from sklearn.metrics import classification_report
import numpy as np

y_pred = model.predict(X_test)

y_pred_classes = np.argmax(y_pred, axis=1)

y_true_classes = np.argmax(y_test, axis=1)

report = classification_report(y_true_classes, y_pred_classes)

print(report)


### Best Result so far!

685/685 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     18117
           1       0.90      0.70      0.79       556
           2       0.96      0.95      0.95      1448
           3       0.82      0.73      0.77       162
           4       0.99      0.99      0.99      1608

    accuracy                           0.98     21891
   macro avg       0.93      0.87      0.90     21891
weighted avg       0.98      0.98      0.98     21891



### With class weight and optimized params:

In [47]:
X_train = mitbih_train_df.iloc[:, :-1]

y_train = mitbih_train_df.iloc[:, -1]

X_test = mitbih_test_df.iloc[:, :-1]
y_test = mitbih_test_df.iloc[:, -1]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(87553, 187) (87553,)
(21891, 187) (21891,)


In [48]:
# Berechne die Class Weights basierend auf der Klassenverteilung
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(y_train),
                                                  y=y_train)

# Konvertiere die Class Weights in ein Dictionary
class_weight_dict = dict(enumerate(class_weights))

print("Class Weights:", class_weight_dict)

Class Weights: {0: 0.2416255002069822, 1: 7.877013045434098, 2: 3.025328265376641, 3: 27.317628705148206, 4: 2.7228424817291246}


In [49]:
from tensorflow.keras.utils import to_categorical

# One-Hot-Encoding der Zielvariablen für Mehrklassenklassifikation
y_train = to_categorical(y_train, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)

In [50]:
# Without batch Normalization: 

from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import class_weight
import numpy as np

# Modell Input
inputs = Input(shape=(187, 1), name="Input")

# 1D-CNN Layer ohne Batch Normalization
first_layer = Conv1D(filters=64, kernel_size=5, activation='relu')(inputs)
third_layer = MaxPooling1D(pool_size=2)(first_layer)

# Zweite CNN-Schicht hinzufügen
fourth_layer = Conv1D(filters=128, kernel_size=7, activation='relu')(third_layer)
sixth_layer = MaxPooling1D(pool_size=2)(fourth_layer)

# LSTM Layer
lstm_layer = LSTM(100, return_sequences=False)(sixth_layer)

# Dense Layers
dense_layer = Dense(100, activation='relu')(lstm_layer)
dropout_layer = Dropout(0.8)(dense_layer)

# Output Layer für mehrklassige Klassifikation
output_layer = Dense(5, activation='softmax')(dropout_layer)

# Modell erstellen
model = Model(inputs=inputs, outputs=output_layer)

# Modell kompilieren
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modell-Übersicht anzeigen
model.summary()

# Early Stopping Callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)


# Modell trainieren mit Class Weights
training_history = model.fit(X_train, y_train, 
                             validation_split=0.2, 
                             epochs=50, 
                             batch_size=16, 
                             class_weight=class_weight_dict,
                             callbacks=[early_stopping])

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input (InputLayer)              │ (None, 187, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_18 (Conv1D)              │ (None, 183, 64)        │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_18 (MaxPooling1D) │ (None, 91, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_19 (Conv1D)              │ (None, 85, 128)        │        57,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_19 (MaxPooling1D) │ (None, 42, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (None, 100)            │        91,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 5)              │           505 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 160,061 (625.24 KB)

 Trainable params: 160,061 (625.24 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 63s 14ms/step - accuracy: 0.2726 - loss: 1.3723 - val_accuracy: 0.3940 - val_loss: 1.3110
Epoch 2/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 62s 14ms/step - accuracy: 0.3554 - loss: 1.1555 - val_accuracy: 0.4513 - val_loss: 1.1887
Epoch 3/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 58s 13ms/step - accuracy: 0.4388 - loss: 0.9474 - val_accuracy: 0.5120 - val_loss: 1.0642
Epoch 4/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 61s 14ms/step - accuracy: 0.5473 - loss: 0.8282 - val_accuracy: 0.7310 - val_loss: 0.7589
Epoch 5/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 58s 13ms/step - accuracy: 0.6431 - loss: 0.6928 - val_accuracy: 0.7974 - val_loss: 0.5898
Epoch 6/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 61s 14ms/step - accuracy: 0.7109 - loss: 0.6303 - val_accuracy: 0.8233 - val_loss: 0.5441
Epoch 7/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 59s 14ms/step - accuracy: 0.7444 - loss: 0.5404 - val_accuracy: 0.7854 - val_loss: 0.6897
Epoch 8/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 60s 14ms/step - accuracy: 0.7661 -

In [51]:
# Prediction: 

from sklearn.metrics import classification_report
import numpy as np

y_pred = model.predict(X_test)

y_pred_classes = np.argmax(y_pred, axis=1)

y_true_classes = np.argmax(y_test, axis=1)

report = classification_report(y_true_classes, y_pred_classes)

print(report)

685/685 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step
              precision    recall  f1-score   support

           0       0.99      0.89      0.93     18117
           1       0.29      0.76      0.42       556
           2       0.73      0.93      0.82      1448
           3       0.23      0.82      0.35       162
           4       0.87      0.98      0.92      1608

    accuracy                           0.89     21891
   macro avg       0.62      0.87      0.69     21891
weighted avg       0.94      0.89      0.91     21891



## **Completely new attempt with Residual CNN:** 

In [55]:
X_train = mitbih_train_df.iloc[:, :-1]

y_train = mitbih_train_df.iloc[:, -1]

X_test = mitbih_test_df.iloc[:, :-1]
y_test = mitbih_test_df.iloc[:, -1]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(87553, 187) (87553,)
(21891, 187) (21891,)


In [56]:
from tensorflow.keras.utils import to_categorical

# One-Hot-Encoding der Zielvariablen für Mehrklassenklassifikation
y_train = to_categorical(y_train, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)

In [10]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Dense, Dropout, Flatten, BatchNormalization, Activation, Add
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Residual Block Definition
def residual_block(x, filters, kernel_size=3):
    # Shortcut Connection
    shortcut = x

    # Adjustment of the shortcut dimensions if required
    if x.shape[-1] != filters:
        shortcut = Conv1D(filters=filters, kernel_size=1, padding='same')(shortcut)
    
    # First Conv1D Layer
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Second Conv1D Layer
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)

    # Adding the shortcut to the output
    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

# Modell Input
inputs = Input(shape=(187, 1), name="Input")

# 1D-CNN Layer
x = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(inputs)
x = MaxPooling1D(pool_size=2)(x)
x = BatchNormalization()(x)

# Residual Block 1
x = residual_block(x, filters=64)

# Residual Block 2
x = residual_block(x, filters=128)

# Residual Block 3
x = residual_block(x, filters=256)

# Flatten Layer
x = Flatten()(x)

# Dense Layers
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)

# Output Layer for multi-class classification
output_layer = Dense(5, activation='softmax')(x)

# Modell erstellen
model = Model(inputs=inputs, outputs=output_layer)

# Modell kompilieren
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Early Stopping Callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Modell Übersicht anzeigen
model.summary()

# Modell trainieren
training_history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, callbacks=[early_stopping])

# Modell evaluieren
evaluation = model.evaluate(X_test, y_test)
print(f"Test loss: {evaluation[0]}, Test accuracy: {evaluation[1]}")

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input (InputLayer)  │ (None, 187, 1)    │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_27 (Conv1D)  │ (None, 187, 64)   │        384 │ Input[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_3     │ (None, 93, 64)    │          0 │ conv1d_27[0][0]   │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 93, 64)    │        256 │ max_pooling1d_3[… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_28 (Conv1D)  │ (None, 93, 64)    │     12,352 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 93, 64)    │        256 │ conv1d_28[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_18       │ (None, 93, 64)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_29 (Conv1D)  │ (None, 93, 64)    │     12,352 │ activation_18[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 93, 64)    │        256 │ conv1d_29[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_9 (Add)         │ (None, 93, 64)    │          0 │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_19       │ (None, 93, 64)    │          0 │ add_9[0][0]       │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_31 (Conv1D)  │ (None, 93, 128)   │     24,704 │ activation_19[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 93, 128)   │        512 │ conv1d_31[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_20       │ (None, 93, 128)   │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_32 (Conv1D)  │ (None, 93, 128)   │     49,280 │ activation_20[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 93, 128)   │        512 │ conv1d_32[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_30 (Conv1D)  │ (None, 93, 128)   │      8,320 │ activation_19[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_10 (Add)        │ (None, 93, 128)   │          0 │ batch_normalizat… │
│                     │                   │            │ conv1d_30[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_21       │ (None, 93, 128)   │          0 │ add_10[0][0]      │
│ (Activation)        │                   │            │                 

 Total params: 3,495,813 (13.34 MB)

 Trainable params: 3,493,893 (13.33 MB)

 Non-trainable params: 1,920 (7.50 KB)

Epoch 1/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 78s 34ms/step - accuracy: 0.9001 - loss: 0.4604 - val_accuracy: 0.9656 - val_loss: 0.1137
Epoch 2/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 75s 34ms/step - accuracy: 0.9619 - loss: 0.1450 - val_accuracy: 0.9722 - val_loss: 0.0955
Epoch 3/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 76s 34ms/step - accuracy: 0.9703 - loss: 0.1138 - val_accuracy: 0.9796 - val_loss: 0.0766
Epoch 4/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 77s 35ms/step - accuracy: 0.9739 - loss: 0.0933 - val_accuracy: 0.9782 - val_loss: 0.0779
Epoch 5/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 80s 37ms/step - accuracy: 0.9779 - loss: 0.0826 - val_accuracy: 0.9824 - val_loss: 0.0619
Epoch 6/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 78s 36ms/step - accuracy: 0.9801 - loss: 0.0746 - val_accuracy: 0.9796 - val_loss: 0.0672
Epoch 7/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 76s 35ms/step - accuracy: 0.9811 - loss: 0.0652 - val_accuracy: 0.9829 - val_loss: 0.0620
Epoch 8/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 77s 35ms/step - accuracy: 0.9825 -

In [11]:
# Prediction: 

from sklearn.metrics import classification_report
import numpy as np

y_pred = model.predict(X_test)

y_pred_classes = np.argmax(y_pred, axis=1)

y_true_classes = np.argmax(y_test, axis=1)

report = classification_report(y_true_classes, y_pred_classes)

print(report)

### Best results so far!

685/685 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     18117
           1       0.93      0.74      0.82       556
           2       0.95      0.96      0.96      1448
           3       0.90      0.65      0.76       162
           4       0.99      0.99      0.99      1608

    accuracy                           0.98     21891
   macro avg       0.95      0.87      0.90     21891
weighted avg       0.98      0.98      0.98     21891



### Residual CNN - With Class Weights

In [61]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Dense, Dropout, Flatten, BatchNormalization, Activation, Add
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Residual Block Definition
def residual_block(x, filters, kernel_size=3):
    # Shortcut Connection
    shortcut = x

    # Anpassung der Shortcut-Dimensionen falls erforderlich
    if x.shape[-1] != filters:
        shortcut = Conv1D(filters=filters, kernel_size=1, padding='same')(shortcut)
    
    # First Conv1D Layer
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Second Conv1D Layer
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)

    # Adding the shortcut to the output
    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

# Modell Input
inputs = Input(shape=(187, 1), name="Input")

# 1D-CNN Layer
x = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(inputs)
x = MaxPooling1D(pool_size=2)(x)
x = BatchNormalization()(x)

# Residual Block 1
x = residual_block(x, filters=64)

# Residual Block 2
x = residual_block(x, filters=128)

# Residual Block 3
x = residual_block(x, filters=256)

# Flatten Layer
x = Flatten()(x)

# Dense Layers
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)

# Output Layer for multi-class classification
output_layer = Dense(5, activation='softmax')(x)

# Modell erstellen
model = Model(inputs=inputs, outputs=output_layer)

# Modell kompilieren
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Early Stopping Callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Modell Übersicht anzeigen
model.summary()

from sklearn.utils import class_weight
import numpy as np

# Berechne die Klassen-Gewichte basierend auf der Häufigkeit der Klassen in y_train
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(np.argmax(y_train, axis=1)),  # Klassen aus One-Hot-Encoding zurückholen
    y=np.argmax(y_train, axis=1))

# Modell trainieren
training_history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, 
                             class_weight=dict(enumerate(class_weights)), 
                             callbacks=[early_stopping])

# Modell evaluieren
evaluation = model.evaluate(X_test, y_test)
print(f"Test loss: {evaluation[0]}, Test accuracy: {evaluation[1]}")

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input (InputLayer)  │ (None, 187, 1)    │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_34 (Conv1D)  │ (None, 187, 64)   │        384 │ Input[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_22    │ (None, 93, 64)    │          0 │ conv1d_34[0][0]   │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 93, 64)    │        256 │ max_pooling1d_22… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_35 (Conv1D)  │ (None, 93, 64)    │     12,352 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 93, 64)    │        256 │ conv1d_35[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_9        │ (None, 93, 64)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_36 (Conv1D)  │ (None, 93, 64)    │     12,352 │ activation_9[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 93, 64)    │        256 │ conv1d_36[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_5 (Add)         │ (None, 93, 64)    │          0 │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_10       │ (None, 93, 64)    │          0 │ add_5[0][0]       │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_38 (Conv1D)  │ (None, 93, 128)   │     24,704 │ activation_10[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 93, 128)   │        512 │ conv1d_38[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_11       │ (None, 93, 128)   │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_39 (Conv1D)  │ (None, 93, 128)   │     49,280 │ activation_11[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 93, 128)   │        512 │ conv1d_39[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_37 (Conv1D)  │ (None, 93, 128)   │      8,320 │ activation_10[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_6 (Add)         │ (None, 93, 128)   │          0 │ batch_normalizat… │
│                     │                   │            │ conv1d_37[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_12       │ (None, 93, 128)   │          0 │ add_6[0][0]       │
│ (Activation)        │                   │            │                 

 Total params: 3,495,813 (13.34 MB)

 Trainable params: 3,493,893 (13.33 MB)

 Non-trainable params: 1,920 (7.50 KB)

Epoch 1/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 207s 80ms/step - accuracy: 0.4629 - loss: 1.5143 - val_accuracy: 0.8226 - val_loss: 0.6151
Epoch 2/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 130s 60ms/step - accuracy: 0.7469 - loss: 0.6220 - val_accuracy: 0.9286 - val_loss: 0.4915
Epoch 3/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 76s 35ms/step - accuracy: 0.8230 - loss: 0.4888 - val_accuracy: 0.8810 - val_loss: 0.4581
Epoch 4/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 83s 38ms/step - accuracy: 0.8495 - loss: 0.4123 - val_accuracy: 0.9171 - val_loss: 0.4121
Epoch 5/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 91s 42ms/step - accuracy: 0.8601 - loss: 0.3814 - val_accuracy: 0.9426 - val_loss: 0.2922
Epoch 6/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 86s 39ms/step - accuracy: 0.8639 - loss: 0.3593 - val_accuracy: 0.9206 - val_loss: 0.3117
Epoch 7/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 86s 39ms/step - accuracy: 0.8606 - loss: 0.3465 - val_accuracy: 0.9557 - val_loss: 0.2043
Epoch 8/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 82s 37ms/step - accuracy: 0.8706

In [62]:
# Prediction: 

from sklearn.metrics import classification_report
import numpy as np

y_pred = model.predict(X_test)

y_pred_classes = np.argmax(y_pred, axis=1)

y_true_classes = np.argmax(y_test, axis=1)

report = classification_report(y_true_classes, y_pred_classes)

print(report)

685/685 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step
              precision    recall  f1-score   support

           0       0.99      0.97      0.98     18117
           1       0.54      0.77      0.64       556
           2       0.90      0.91      0.91      1448
           3       0.46      0.85      0.60       162
           4       0.97      0.97      0.97      1608

    accuracy                           0.96     21891
   macro avg       0.77      0.89      0.82     21891
weighted avg       0.96      0.96      0.96     21891



### Combination of Residual CNN and LSTM: 

In [63]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Dense, Dropout, Flatten, BatchNormalization, Activation, Add, LSTM
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Residual Block Definition
def residual_block(x, filters, kernel_size=3):
    # Shortcut Connection
    shortcut = x

    # Anpassung der Shortcut-Dimensionen falls erforderlich
    if x.shape[-1] != filters:
        shortcut = Conv1D(filters=filters, kernel_size=1, padding='same')(shortcut)
    
    # First Conv1D Layer
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Second Conv1D Layer
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)

    # Adding the shortcut to the output
    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

# Modell Input
inputs = Input(shape=(187, 1), name="Input")

# 1D-CNN Layer
x = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(inputs)
x = MaxPooling1D(pool_size=2)(x)
x = BatchNormalization()(x)

# Residual Block 1
x = residual_block(x, filters=64)

# Residual Block 2
x = residual_block(x, filters=128)

# Residual Block 3
x = residual_block(x, filters=256)

# LSTM Layer nach den Residual Blocks
x = LSTM(100, return_sequences=False)(x)

# Dense Layers
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)

# Output Layer for multi-class classification
output_layer = Dense(5, activation='softmax')(x)

# Modell erstellen
model = Model(inputs=inputs, outputs=output_layer)

# Modell kompilieren
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Early Stopping Callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Modell Übersicht anzeigen
model.summary()

# Modell trainieren
training_history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, callbacks=[early_stopping])

# Modell evaluieren
evaluation = model.evaluate(X_test, y_test)
print(f"Test loss: {evaluation[0]}, Test accuracy: {evaluation[1]}")


Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input (InputLayer)  │ (None, 187, 1)    │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_43 (Conv1D)  │ (None, 187, 64)   │        384 │ Input[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_23    │ (None, 93, 64)    │          0 │ conv1d_43[0][0]   │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 93, 64)    │        256 │ max_pooling1d_23… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_44 (Conv1D)  │ (None, 93, 64)    │     12,352 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 93, 64)    │        256 │ conv1d_44[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_15       │ (None, 93, 64)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_45 (Conv1D)  │ (None, 93, 64)    │     12,352 │ activation_15[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 93, 64)    │        256 │ conv1d_45[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_8 (Add)         │ (None, 93, 64)    │          0 │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_16       │ (None, 93, 64)    │          0 │ add_8[0][0]       │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_47 (Conv1D)  │ (None, 93, 128)   │     24,704 │ activation_16[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 93, 128)   │        512 │ conv1d_47[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_17       │ (None, 93, 128)   │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_48 (Conv1D)  │ (None, 93, 128)   │     49,280 │ activation_17[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 93, 128)   │        512 │ conv1d_48[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_46 (Conv1D)  │ (None, 93, 128)   │      8,320 │ activation_16[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_9 (Add)         │ (None, 93, 128)   │          0 │ batch_normalizat… │
│                     │                   │            │ conv1d_46[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_18       │ (None, 93, 128)   │          0 │ add_9[0][0]       │
│ (Activation)        │                   │            │                 

 Total params: 603,989 (2.30 MB)

 Trainable params: 602,069 (2.30 MB)

 Non-trainable params: 1,920 (7.50 KB)

Epoch 1/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 240s 108ms/step - accuracy: 0.8283 - loss: 0.6428 - val_accuracy: 0.9015 - val_loss: 0.3863
Epoch 2/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 243s 111ms/step - accuracy: 0.9359 - loss: 0.2472 - val_accuracy: 0.9553 - val_loss: 0.1758
Epoch 3/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 157s 72ms/step - accuracy: 0.9566 - loss: 0.1686 - val_accuracy: 0.9629 - val_loss: 0.1441
Epoch 4/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 113s 52ms/step - accuracy: 0.9614 - loss: 0.1467 - val_accuracy: 0.9618 - val_loss: 0.1441
Epoch 5/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 120s 55ms/step - accuracy: 0.9665 - loss: 0.1264 - val_accuracy: 0.9651 - val_loss: 0.1376
Epoch 6/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 118s 54ms/step - accuracy: 0.9712 - loss: 0.1128 - val_accuracy: 0.9680 - val_loss: 0.1273
Epoch 7/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 114s 52ms/step - accuracy: 0.9733 - loss: 0.1014 - val_accuracy: 0.9706 - val_loss: 0.1084
Epoch 8/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 122s 56ms/step - accuracy

In [64]:
# Prediction: 

from sklearn.metrics import classification_report
import numpy as np

y_pred = model.predict(X_test)

y_pred_classes = np.argmax(y_pred, axis=1)

y_true_classes = np.argmax(y_test, axis=1)

report = classification_report(y_true_classes, y_pred_classes)

print(report)

685/685 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     18117
           1       0.93      0.64      0.76       556
           2       0.96      0.96      0.96      1448
           3       0.87      0.67      0.76       162
           4       0.99      0.98      0.99      1608

    accuracy                           0.98     21891
   macro avg       0.95      0.85      0.89     21891
weighted avg       0.98      0.98      0.98     21891



### Residual CNN With LR-Scheduler: 

In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Dense, Dropout, Flatten, BatchNormalization, Activation, Add
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Residual Block Definition
def residual_block(x, filters, kernel_size=3):
    # Shortcut Connection
    shortcut = x

    # Anpassung der Shortcut-Dimensionen falls erforderlich
    if x.shape[-1] != filters:
        shortcut = Conv1D(filters=filters, kernel_size=1, padding='same')(shortcut)
    
    # First Conv1D Layer
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Second Conv1D Layer
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)

    # Adding the shortcut to the output
    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

# Modell Input
inputs = Input(shape=(187, 1), name="Input")

# 1D-CNN Layer
x = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(inputs)
x = MaxPooling1D(pool_size=2)(x)
x = BatchNormalization()(x)

# Residual Block 1
x = residual_block(x, filters=64)

# Residual Block 2
x = residual_block(x, filters=128)

# Residual Block 3
x = residual_block(x, filters=256)

# Flatten Layer
x = Flatten()(x)

# Dense Layers
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)

# Output Layer for multi-class classification
output_layer = Dense(5, activation='softmax')(x)

# Modell erstellen
model = Model(inputs=inputs, outputs=output_layer)

# Modell kompilieren
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Early Stopping Callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# LR-Scheduler
lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-6)

# Modell Übersicht anzeigen
model.summary()

# Modell trainieren
training_history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, callbacks=[early_stopping, lr_scheduler])

# Modell evaluieren
evaluation = model.evaluate(X_test, y_test)
print(f"Test loss: {evaluation[0]}, Test accuracy: {evaluation[1]}")

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input (InputLayer)  │ (None, 187, 1)    │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_18 (Conv1D)  │ (None, 187, 64)   │        384 │ Input[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_2     │ (None, 93, 64)    │          0 │ conv1d_18[0][0]   │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 93, 64)    │        256 │ max_pooling1d_2[… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_19 (Conv1D)  │ (None, 93, 64)    │     12,352 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 93, 64)    │        256 │ conv1d_19[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_12       │ (None, 93, 64)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_20 (Conv1D)  │ (None, 93, 64)    │     12,352 │ activation_12[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 93, 64)    │        256 │ conv1d_20[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_6 (Add)         │ (None, 93, 64)    │          0 │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_13       │ (None, 93, 64)    │          0 │ add_6[0][0]       │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_22 (Conv1D)  │ (None, 93, 128)   │     24,704 │ activation_13[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 93, 128)   │        512 │ conv1d_22[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_14       │ (None, 93, 128)   │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_23 (Conv1D)  │ (None, 93, 128)   │     49,280 │ activation_14[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 93, 128)   │        512 │ conv1d_23[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_21 (Conv1D)  │ (None, 93, 128)   │      8,320 │ activation_13[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_7 (Add)         │ (None, 93, 128)   │          0 │ batch_normalizat… │
│                     │                   │            │ conv1d_21[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_15       │ (None, 93, 128)   │          0 │ add_7[0][0]       │
│ (Activation)        │                   │            │                 

 Total params: 3,495,813 (13.34 MB)

 Trainable params: 3,493,893 (13.33 MB)

 Non-trainable params: 1,920 (7.50 KB)

Epoch 1/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 90s 39ms/step - accuracy: 0.9015 - loss: 0.4346 - val_accuracy: 0.9677 - val_loss: 0.1283 - learning_rate: 0.0010
Epoch 2/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 171s 78ms/step - accuracy: 0.9620 - loss: 0.1464 - val_accuracy: 0.9768 - val_loss: 0.0912 - learning_rate: 0.0010
Epoch 3/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 234s 107ms/step - accuracy: 0.9720 - loss: 0.1065 - val_accuracy: 0.9757 - val_loss: 0.0883 - learning_rate: 0.0010
Epoch 4/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 244s 111ms/step - accuracy: 0.9759 - loss: 0.0919 - val_accuracy: 0.9810 - val_loss: 0.0690 - learning_rate: 0.0010
Epoch 5/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 236s 108ms/step - accuracy: 0.9774 - loss: 0.0830 - val_accuracy: 0.9833 - val_loss: 0.0604 - learning_rate: 0.0010
Epoch 6/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 238s 109ms/step - accuracy: 0.9809 - loss: 0.0702 - val_accuracy: 0.9820 - val_loss: 0.0679 - learning_rate: 0.0010
Epoch 7/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 241s 110ms/st

In [13]:
# Prediction: 

from sklearn.metrics import classification_report
import numpy as np

y_pred = model.predict(X_test)

y_pred_classes = np.argmax(y_pred, axis=1)

y_true_classes = np.argmax(y_test, axis=1)

report = classification_report(y_true_classes, y_pred_classes)

print(report)

### Precision dropped a little but recall improved. 

685/685 ━━━━━━━━━━━━━━━━━━━━ 28s 40ms/step
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     18117
           1       0.95      0.67      0.79       556
           2       0.96      0.94      0.95      1448
           3       0.82      0.71      0.76       162
           4       0.99      0.99      0.99      1608

    accuracy                           0.98     21891
   macro avg       0.94      0.86      0.90     21891
weighted avg       0.98      0.98      0.98     21891



###  Residual CNN With reduced Batch size

In [12]:
# Residual Block Definition
def residual_block(x, filters, kernel_size=3):
    # Shortcut Connection
    shortcut = x

    # Anpassung der Shortcut-Dimensionen falls erforderlich
    if x.shape[-1] != filters:
        shortcut = Conv1D(filters=filters, kernel_size=1, padding='same')(shortcut)
    
    # First Conv1D Layer
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Second Conv1D Layer
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)

    # Adding the shortcut to the output
    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

# Modell Input
inputs = Input(shape=(187, 1), name="Input")

# 1D-CNN Layer
x = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(inputs)
x = MaxPooling1D(pool_size=2)(x)
x = BatchNormalization()(x)

# Residual Block 1
x = residual_block(x, filters=64)

# Residual Block 2
x = residual_block(x, filters=128)

# Residual Block 3
x = residual_block(x, filters=256)

# Flatten Layer
x = Flatten()(x)

# Dense Layers
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)

# Output Layer for multi-class classification
output_layer = Dense(5, activation='softmax')(x)

# Modell erstellen
model = Model(inputs=inputs, outputs=output_layer)

# Modell kompilieren
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Early Stopping Callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Modell Übersicht anzeigen
model.summary()

# Modell trainieren
training_history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=16, callbacks=[early_stopping])

# Modell evaluieren
evaluation = model.evaluate(X_test, y_test)
print(f"Test loss: {evaluation[0]}, Test accuracy: {evaluation[1]}")

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input (InputLayer)  │ (None, 187, 1)    │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_36 (Conv1D)  │ (None, 187, 64)   │        384 │ Input[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_4     │ (None, 93, 64)    │          0 │ conv1d_36[0][0]   │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 93, 64)    │        256 │ max_pooling1d_4[… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_37 (Conv1D)  │ (None, 93, 64)    │     12,352 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 93, 64)    │        256 │ conv1d_37[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_24       │ (None, 93, 64)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_38 (Conv1D)  │ (None, 93, 64)    │     12,352 │ activation_24[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 93, 64)    │        256 │ conv1d_38[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_12 (Add)        │ (None, 93, 64)    │          0 │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_25       │ (None, 93, 64)    │          0 │ add_12[0][0]      │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_40 (Conv1D)  │ (None, 93, 128)   │     24,704 │ activation_25[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 93, 128)   │        512 │ conv1d_40[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_26       │ (None, 93, 128)   │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_41 (Conv1D)  │ (None, 93, 128)   │     49,280 │ activation_26[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 93, 128)   │        512 │ conv1d_41[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_39 (Conv1D)  │ (None, 93, 128)   │      8,320 │ activation_25[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_13 (Add)        │ (None, 93, 128)   │          0 │ batch_normalizat… │
│                     │                   │            │ conv1d_39[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_27       │ (None, 93, 128)   │          0 │ add_13[0][0]      │
│ (Activation)        │                   │            │                 

 Total params: 3,495,813 (13.34 MB)

 Trainable params: 3,493,893 (13.33 MB)

 Non-trainable params: 1,920 (7.50 KB)

Epoch 1/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 227s 51ms/step - accuracy: 0.8988 - loss: 0.4289 - val_accuracy: 0.9693 - val_loss: 0.1126
Epoch 2/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 264s 60ms/step - accuracy: 0.9646 - loss: 0.1386 - val_accuracy: 0.9689 - val_loss: 0.1052
Epoch 3/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 280s 64ms/step - accuracy: 0.9698 - loss: 0.1143 - val_accuracy: 0.9750 - val_loss: 0.0879
Epoch 4/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 107s 24ms/step - accuracy: 0.9731 - loss: 0.1007 - val_accuracy: 0.9766 - val_loss: 0.1083
Epoch 5/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 248s 57ms/step - accuracy: 0.9764 - loss: 0.0892 - val_accuracy: 0.9789 - val_loss: 0.0781
Epoch 6/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 133s 30ms/step - accuracy: 0.9786 - loss: 0.0787 - val_accuracy: 0.9793 - val_loss: 0.0740
Epoch 7/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 169s 39ms/step - accuracy: 0.9800 - loss: 0.0769 - val_accuracy: 0.9806 - val_loss: 0.0721
Epoch 8/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 204s 46ms/step - accuracy: 

In [14]:
# Prediction: 

from sklearn.metrics import classification_report
import numpy as np

y_pred = model.predict(X_test)

y_pred_classes = np.argmax(y_pred, axis=1)

y_true_classes = np.argmax(y_test, axis=1)

report = classification_report(y_true_classes, y_pred_classes)

print(report)

685/685 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     18117
           1       0.95      0.67      0.79       556
           2       0.96      0.94      0.95      1448
           3       0.82      0.71      0.76       162
           4       0.99      0.99      0.99      1608

    accuracy                           0.98     21891
   macro avg       0.94      0.86      0.90     21891
weighted avg       0.98      0.98      0.98     21891



### CNN with L2 Regularization

### CNN with Adasyn

In [ ]:
# Verwende ADASYN, um den Trainingsdatensatz zu balancieren
adasyn = ADASYN(sampling_strategy='minority')

# Annahme: X_train und y_train enthalten die Originaldaten
# Stelle sicher, dass du den Datensatz entsprechend vorverarbeitet hast
X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train.reshape(-1, 187), y_train)

# Reshape für das CNN-Modell (weil CNN eine 3D-Eingabe erwartet)
X_train_resampled = X_train_resampled.reshape(-1, 187, 1)

# Modell trainieren
training_history = model.fit(X_train_resampled, y_train_resampled, validation_split=0.2, epochs=50, batch_size=32, callbacks=[early_stopping])

# Modell evaluieren
evaluation = model.evaluate(X_test, y_test)
print(f"Test loss: {evaluation[0]}, Test accuracy: {evaluation[1]}")

In [ ]:
# Prediction: 

from sklearn.metrics import classification_report
import numpy as np

y_pred = model.predict(X_test)

y_pred_classes = np.argmax(y_pred, axis=1)

y_true_classes = np.argmax(y_test, axis=1)

report = classification_report(y_true_classes, y_pred_classes)

print(report)

### CNN with Data Augmentation

In [ ]:
# Prediction: 

from sklearn.metrics import classification_report
import numpy as np

y_pred = model.predict(X_test)

y_pred_classes = np.argmax(y_pred, axis=1)

y_true_classes = np.argmax(y_test, axis=1)

report = classification_report(y_true_classes, y_pred_classes)

print(report)

### CNN- Optimization with Keras-Tuner

In [8]:
import keras_tuner as kt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Dense, Dropout, Flatten, BatchNormalization, Activation, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Residual Block Definition
def residual_block(x, filters, kernel_size=3):
    # Shortcut Connection
    shortcut = x

    # Anpassung der Shortcut-Dimensionen falls erforderlich
    if x.shape[-1] != filters:
        shortcut = Conv1D(filters=filters, kernel_size=1, padding='same')(shortcut)
    
    # First Conv1D Layer
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Second Conv1D Layer
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)

    # Adding the shortcut to the output
    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

# Modell-Builder Funktion für den Keras Tuner
def build_model(hp):
    inputs = Input(shape=(187, 1), name="Input")
    
    # 1D-CNN Layer mit Hyperparameter für Filteranzahl und Kernel-Größe
    x = Conv1D(
        filters=hp.Int('filters', min_value=32, max_value=256, step=32),
        kernel_size=hp.Int('kernel_size', min_value=3, max_value=7, step=2),
        activation='relu',
        padding='same'
    )(inputs)
    x = MaxPooling1D(pool_size=2)(x)
    x = BatchNormalization()(x)

    # Residual Block 1
    x = residual_block(x, filters=hp.Int('filters', min_value=32, max_value=256, step=32))

    # Residual Block 2
    x = residual_block(x, filters=hp.Int('filters', min_value=32, max_value=256, step=32))

    # Residual Block 3
    x = residual_block(x, filters=hp.Int('filters', min_value=32, max_value=256, step=32))

    # Flatten Layer
    x = Flatten()(x)

    # Dense Layers mit Hyperparameter für Dropout-Werte
    x = Dense(128, activation='relu')(x)
    x = Dropout(hp.Float('dropout_1', min_value=0.3, max_value=0.6, step=0.1))(x)

    x = Dense(64, activation='relu')(x)
    x = Dropout(hp.Float('dropout_2', min_value=0.3, max_value=0.6, step=0.1))(x)

    # Output Layer for multi-class classification
    output_layer = Dense(5, activation='softmax')(x)

    # Modell erstellen
    model = Model(inputs=inputs, outputs=output_layer)

    # Modell kompilieren mit variabler Lernrate
    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Keras Tuner Hyperband für die Optimierung
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',  # Ziel der Optimierung
    max_epochs=50,
    factor=3,
    directory='keras_tuner_dir',  # Speicherort für die Ergebnisse
    project_name='ekg_classification_optimization'
)

# Callback für Early Stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Hyperparameter-Suche durchführen
tuner.search(X_train, y_train, validation_split=0.2, epochs=30, batch_size=32, callbacks=[early_stopping])

# Beste Hyperparameter anzeigen
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
Beste Lernrate: {best_hps.get('learning_rate')}
Beste Filteranzahl: {best_hps.get('filters')}
Beste Kernel-Größe: {best_hps.get('kernel_size')}
Bestes Dropout für Layer 1: {best_hps.get('dropout_1')}
Bestes Dropout für Layer 2: {best_hps.get('dropout_2')}
""")

# Bestes Modell mit den besten Hyperparametern trainieren
best_model = tuner.hypermodel.build(best_hps)
training_history = best_model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, callbacks=[early_stopping])

# Modell evaluieren
evaluation = best_model.evaluate(X_test, y_test)
print(f"Test loss: {evaluation[0]}, Test accuracy: {evaluation[1]}")


Reloading Tuner from keras_tuner_dir\ekg_classification_optimization\tuner0.json

Beste Lernrate: 0.0001
Beste Filteranzahl: 64
Beste Kernel-Größe: 3
Bestes Dropout für Layer 1: 0.4
Bestes Dropout für Layer 2: 0.3

Epoch 1/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 52s 22ms/step - accuracy: 0.8963 - loss: 0.3814 - val_accuracy: 0.9640 - val_loss: 0.1218
Epoch 2/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 64s 29ms/step - accuracy: 0.9624 - loss: 0.1383 - val_accuracy: 0.9733 - val_loss: 0.0944
Epoch 3/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 106s 48ms/step - accuracy: 0.9684 - loss: 0.1150 - val_accuracy: 0.9790 - val_loss: 0.0726
Epoch 4/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 102s 47ms/step - accuracy: 0.9735 - loss: 0.0951 - val_accuracy: 0.9787 - val_loss: 0.0723
Epoch 5/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 55s 25ms/step - accuracy: 0.9767 - loss: 0.0832 - val_accuracy: 0.9818 - val_loss: 0.0615
Epoch 6/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 65s 29ms/step - accuracy: 0.9796 - loss: 0.0719 - val_accuracy: 0.9838 - val_loss

In [10]:
# Prediction: 

from sklearn.metrics import classification_report
import numpy as np

y_pred = best_model.predict(X_test)

y_pred_classes = np.argmax(y_pred, axis=1)

y_true_classes = np.argmax(y_test, axis=1)

report = classification_report(y_true_classes, y_pred_classes)

print(report)

685/685 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     18117
           1       0.94      0.74      0.83       556
           2       0.95      0.97      0.96      1448
           3       0.90      0.67      0.77       162
           4       1.00      0.99      0.99      1608

    accuracy                           0.98     21891
   macro avg       0.96      0.87      0.91     21891
weighted avg       0.98      0.98      0.98     21891



### Best Results so far: 

In [5]:
X_train = mitbih_train_df.iloc[:, :-1]  # Alle Spalten außer der letzten (Label)
y_train = mitbih_train_df.iloc[:, -1]   # Letzte Spalte (Label)

X_test = mitbih_test_df.iloc[:, :-1]
y_test = mitbih_test_df.iloc[:, -1]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

from tensorflow.keras.utils import to_categorical

# One-Hot-Encoding der Zielvariablen für Mehrklassenklassifikation
y_train = to_categorical(y_train, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)

# Umwandlung von DataFrames in NumPy-Arrays und Hinzufügen einer Feature-Dimension
X_train = np.expand_dims(X_train.values, axis=-1)
X_test = np.expand_dims(X_test.values, axis=-1)

print("Shape von X_train:", X_train.shape)  # Sollte (Samples, Timesteps, Features) sein
print("Shape von X_test:", X_test.shape)

(87553, 187) (87553,)
(21891, 187) (21891,)
Shape von X_train: (87553, 187, 1)
Shape von X_test: (21891, 187, 1)


In [6]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Dense, Dropout, Flatten, BatchNormalization, Activation, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Residual Block Definition
def residual_block(x, filters, kernel_size=3):
    # Shortcut Connection
    shortcut = x

    # Anpassung der Shortcut-Dimensionen falls erforderlich
    if x.shape[-1] != filters:
        shortcut = Conv1D(filters=filters, kernel_size=1, padding='same')(shortcut)
    
    # First Conv1D Layer
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Second Conv1D Layer
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)

    # Adding the shortcut to the output
    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

# Modell Input
inputs = Input(shape=(187, 1), name="Input")

# 1D-CNN Layer mit den besten Hyperparametern
x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inputs)
x = MaxPooling1D(pool_size=2)(x)
x = BatchNormalization()(x)

# Residual Block 1
x = residual_block(x, filters=64)

# Residual Block 2
x = residual_block(x, filters=128)

# Residual Block 3
x = residual_block(x, filters=256)

# Flatten Layer
x = Flatten()(x)

# Dense Layers mit den besten Dropout-Werten
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)  # Best Dropout für Layer 1

x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)  # Best Dropout für Layer 2

# Output Layer für Mehrklassen-Klassifikation
output_layer = Dense(5, activation='softmax')(x)

# Modell erstellen
model = Model(inputs=inputs, outputs=output_layer)

# Modell kompilieren mit der besten Lernrate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Early Stopping Callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Modell trainieren
training_history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, callbacks=[early_stopping])

# Modell evaluieren
evaluation = model.evaluate(X_test, y_test)
print(f"Test loss: {evaluation[0]}, Test accuracy: {evaluation[1]}")

Epoch 1/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 87s 38ms/step - accuracy: 0.9140 - loss: 0.3124 - val_accuracy: 0.9706 - val_loss: 0.1163
Epoch 2/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 90s 41ms/step - accuracy: 0.9687 - loss: 0.1192 - val_accuracy: 0.9810 - val_loss: 0.0692
Epoch 3/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 103s 47ms/step - accuracy: 0.9751 - loss: 0.0895 - val_accuracy: 0.9844 - val_loss: 0.0565
Epoch 4/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 93s 42ms/step - accuracy: 0.9802 - loss: 0.0711 - val_accuracy: 0.9845 - val_loss: 0.0564
Epoch 5/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 98s 45ms/step - accuracy: 0.9841 - loss: 0.0571 - val_accuracy: 0.9861 - val_loss: 0.0530
Epoch 6/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 86s 39ms/step - accuracy: 0.9850 - loss: 0.0515 - val_accuracy: 0.9863 - val_loss: 0.0502
Epoch 7/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 98s 45ms/step - accuracy: 0.9872 - loss: 0.0435 - val_accuracy: 0.9867 - val_loss: 0.0478
Epoch 8/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 89s 41ms/step - accuracy: 0.9892 

In [7]:
# Prediction: 

from sklearn.metrics import classification_report
import numpy as np

y_pred = model.predict(X_test)

y_pred_classes = np.argmax(y_pred, axis=1)

y_true_classes = np.argmax(y_test, axis=1)

report = classification_report(y_true_classes, y_pred_classes)

print(report)

685/685 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     18117
           1       0.94      0.81      0.87       556
           2       0.98      0.94      0.96      1448
           3       0.82      0.78      0.80       162
           4       1.00      0.98      0.99      1608

    accuracy                           0.99     21891
   macro avg       0.94      0.90      0.92     21891
weighted avg       0.99      0.99      0.99     21891



#### **With SGD Optimizer:**

In [12]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Dense, Dropout, Flatten, BatchNormalization, Activation, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD

# Residual Block Definition
def residual_block(x, filters, kernel_size=3):
    # Shortcut Connection
    shortcut = x

    # Anpassung der Shortcut-Dimensionen falls erforderlich
    if x.shape[-1] != filters:
        shortcut = Conv1D(filters=filters, kernel_size=1, padding='same')(shortcut)
    
    # First Conv1D Layer
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Second Conv1D Layer
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)

    # Adding the shortcut to the output
    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

# Modell Input
inputs = Input(shape=(187, 1), name="Input")

# 1D-CNN Layer mit den besten Hyperparametern
x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inputs)
x = MaxPooling1D(pool_size=2)(x)
x = BatchNormalization()(x)

# Residual Block 1
x = residual_block(x, filters=64)

# Residual Block 2
x = residual_block(x, filters=128)

# Residual Block 3
x = residual_block(x, filters=256)

# Flatten Layer
x = Flatten()(x)

# Dense Layers mit den besten Dropout-Werten
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)  # Best Dropout für Layer 1

x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)  # Best Dropout für Layer 2

# Output Layer für Mehrklassen-Klassifikation
output_layer = Dense(5, activation='softmax')(x)

# Modell erstellen
model = Model(inputs=inputs, outputs=output_layer)

# Modell kompilieren mit der besten Lernrate
model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.97), loss='categorical_crossentropy', metrics=['accuracy'])

# Early Stopping Callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Modell trainieren
training_history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, callbacks=[early_stopping])

# Modell evaluieren
evaluation = model.evaluate(X_test, y_test)
print(f"Test loss: {evaluation[0]}, Test accuracy: {evaluation[1]}")

Epoch 1/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 83s 37ms/step - accuracy: 0.8879 - loss: 0.3878 - val_accuracy: 0.9645 - val_loss: 0.1336
Epoch 2/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 95s 44ms/step - accuracy: 0.9579 - loss: 0.1511 - val_accuracy: 0.9713 - val_loss: 0.1001
Epoch 3/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 94s 43ms/step - accuracy: 0.9688 - loss: 0.1181 - val_accuracy: 0.9736 - val_loss: 0.0930
Epoch 4/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 91s 42ms/step - accuracy: 0.9732 - loss: 0.0975 - val_accuracy: 0.9772 - val_loss: 0.0856
Epoch 5/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 99s 45ms/step - accuracy: 0.9758 - loss: 0.0890 - val_accuracy: 0.9793 - val_loss: 0.0725
Epoch 6/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 88s 40ms/step - accuracy: 0.9771 - loss: 0.0794 - val_accuracy: 0.9817 - val_loss: 0.0645
Epoch 7/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 100s 46ms/step - accuracy: 0.9794 - loss: 0.0713 - val_accuracy: 0.9829 - val_loss: 0.0652
Epoch 8/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 87s 40ms/step - accuracy: 0.9815 

In [ ]:
# Prediction: momentum == 0.95

from sklearn.metrics import classification_report
import numpy as np

y_pred = model.predict(X_test)

y_pred_classes = np.argmax(y_pred, axis=1)

y_true_classes = np.argmax(y_test, axis=1)

report = classification_report(y_true_classes, y_pred_classes)

print(report)

685/685 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     18117
           1       0.92      0.77      0.84       556
           2       0.98      0.95      0.96      1448
           3       0.86      0.77      0.81       162
           4       1.00      0.99      0.99      1608

    accuracy                           0.99     21891
   macro avg       0.95      0.89      0.92     21891
weighted avg       0.99      0.99      0.99     21891



In [ ]:
# Prediction: momentum == 0.96

from sklearn.metrics import classification_report
import numpy as np

y_pred = model.predict(X_test)

y_pred_classes = np.argmax(y_pred, axis=1)

y_true_classes = np.argmax(y_test, axis=1)

report = classification_report(y_true_classes, y_pred_classes)

print(report)

685/685 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     18117
           1       0.95      0.74      0.83       556
           2       0.95      0.96      0.96      1448
           3       0.90      0.70      0.79       162
           4       0.99      0.99      0.99      1608

    accuracy                           0.98     21891
   macro avg       0.96      0.88      0.91     21891
weighted avg       0.98      0.98      0.98     21891



In [14]:
# Prediction: momentum == 0.97

from sklearn.metrics import classification_report
import numpy as np

y_pred = model.predict(X_test)

y_pred_classes = np.argmax(y_pred, axis=1)

y_true_classes = np.argmax(y_test, axis=1)

report = classification_report(y_true_classes, y_pred_classes)

print(report)


685/685 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     18117
           1       0.89      0.81      0.85       556
           2       0.99      0.94      0.96      1448
           3       0.81      0.77      0.79       162
           4       0.99      0.99      0.99      1608

    accuracy                           0.99     21891
   macro avg       0.93      0.90      0.92     21891
weighted avg       0.99      0.99      0.99     21891



With manual Class Weights: 

In [20]:
import numpy as np
from tensorflow.keras.utils import to_categorical

# Verwendung der vorverarbeiteten DataFrames
X_train = mitbih_train_df.iloc[:, :-1]  # Alle Spalten außer der letzten (Label)
y_train = mitbih_train_df.iloc[:, -1]   # Letzte Spalte (Label)

X_test = mitbih_test_df.iloc[:, :-1]
y_test = mitbih_test_df.iloc[:, -1]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

# Manuelle Festlegung der Klassengewichte
class_weights = {
    0: 1,   # Klasse 0
    1: 5,  # Klasse 1
    2: 2,   # Klasse 2
    3: 15,  # Klasse 3
    4: 2    # Klasse 4
}

print("Manuell festgelegte Klassengewichte:", class_weights)

# Umwandlung von DataFrames in NumPy-Arrays und Hinzufügen einer Feature-Dimension
X_train = np.expand_dims(X_train.values, axis=-1)
X_test = np.expand_dims(X_test.values, axis=-1)

print("Shape von X_train:", X_train.shape)  # Sollte (Samples, Timesteps, Features) sein
print("Shape von X_test:", X_test.shape)

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Dense, Dropout, Flatten, BatchNormalization, Activation, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Residual Block
def residual_block(x, filters, kernel_size=3):
    # Shortcut-Verbindung
    shortcut = x

    if x.shape[-1] != filters:
        shortcut = Conv1D(filters=filters, kernel_size=1, padding='same')(shortcut)
    
    # Erste Conv1D-Schicht
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Zweite Conv1D-Schicht
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)

    # Hinzufügen der Shortcut-Verbindung
    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

# Anzahl der Timesteps aus den Daten bestimmen
timesteps = X_train.shape[1]

# Modellinput
inputs = Input(shape=(timesteps, 1), name="Input")

# 1D-CNN-Schicht mit den besten Hyperparametern
x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inputs)
x = MaxPooling1D(pool_size=2)(x)
x = BatchNormalization()(x)

# Residual Block 1
x = residual_block(x, filters=64)

# Residual Block 2
x = residual_block(x, filters=128)

# Residual Block 3
x = residual_block(x, filters=256)

# Flatten-Schicht
x = Flatten()(x)

# Dense-Schichten mit den besten Dropout-Werten
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)  # Bestes Dropout für Layer 1

x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)  # Bestes Dropout für Layer 2

# Output-Schicht für Mehrklassenklassifikation
output_layer = Dense(5, activation='softmax')(x)

# Modell erstellen
model = Model(inputs=inputs, outputs=output_layer)

# Modell kompilieren mit der besten Lernrate und angepasster Loss-Funktion
model.compile(optimizer = Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# Early Stopping Callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Modell trainieren mit manuellen Klassengewichten
training_history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping],
    class_weight=class_weights
)

# Modell evaluieren
evaluation = model.evaluate(X_test, y_test)
print(f"Test loss: {evaluation[0]}, Test accuracy: {evaluation[1]}")

# Vorhersage

from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

report = classification_report(y_test, y_pred_classes)
print(report)

(87553, 187) (87553,)
(21891, 187) (21891,)
Manuell festgelegte Klassengewichte: {0: 1, 1: 5, 2: 2, 3: 15, 4: 2}
Shape von X_train: (87553, 187, 1)
Shape von X_test: (21891, 187, 1)
Epoch 1/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 104s 46ms/step - accuracy: 0.8696 - loss: 0.9343 - val_accuracy: 0.9597 - val_loss: 0.1573
Epoch 2/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 100s 46ms/step - accuracy: 0.9406 - loss: 0.4535 - val_accuracy: 0.9716 - val_loss: 0.1018
Epoch 3/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 109s 50ms/step - accuracy: 0.9580 - loss: 0.3158 - val_accuracy: 0.9663 - val_loss: 0.1237
Epoch 4/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 102s 47ms/step - accuracy: 0.9653 - loss: 0.2533 - val_accuracy: 0.9734 - val_loss: 0.0985
Epoch 5/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 113s 52ms/step - accuracy: 0.9710 - loss: 0.2302 - val_accuracy: 0.9761 - val_loss: 0.0933
Epoch 6/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 106s 48ms/step - accuracy: 0.9733 - loss: 0.1929 - val_accuracy: 0.9718 - val_loss: 0.0978
Epoch 7/50
2189/2189

#### **With Own Loss Function**

In [29]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K

# Klassenfrequenzen
class_counts = {
    0: 72470,
    1: 2223,
    2: 5788,
    3: 641,
    4: 6431
}

total_samples = sum(class_counts.values())
num_classes = len(class_counts)

# Berechnung der Klassengewichte
class_weights = {}

for class_id, count in class_counts.items():
    # Invers proportional zur Klassenhäufigkeit
    class_weight = total_samples / (num_classes * count)
    class_weights[class_id] = class_weight

print("Klassengewichte:", class_weights)

def weighted_categorical_crossentropy(weights):
    """
    Gewichtete Version der kategorialen Kreuzentropie.
    Parameter:
        weights: Array oder Liste der Klassengewichte in der Reihenfolge der Klassenlabels.
    Rückgabe:
        Verlustfunktion, die (y_true, y_pred) als Eingaben akzeptiert.
    """
    weights = K.variable(weights)

    def loss(y_true, y_pred):
        # Clipping von y_pred, um Division durch Null zu vermeiden
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # Berechnung der Kreuzentropie
        cross_entropy = y_true * K.log(y_pred)
        # Gewichtung der Kreuzentropie
        weights_per_sample = K.sum(weights * y_true, axis=-1)
        weighted_cross_entropy = -K.sum(cross_entropy, axis=-1) * weights_per_sample
        return weighted_cross_entropy

    return loss

# Umwandlung der Klassengewichte in ein Array entsprechend der Klassenreihenfolge
weights_array = np.array([class_weights[i] for i in range(num_classes)])

# Erstellung der Verlustfunktion
loss_function = weighted_categorical_crossentropy(weights_array)

########################################################

# Verwendung der vorverarbeiteten DataFrames
X_train = mitbih_train_df.iloc[:, :-1]  # Alle Spalten außer der letzten (Label)
y_train = mitbih_train_df.iloc[:, -1]   # Letzte Spalte (Label)

X_test = mitbih_test_df.iloc[:, :-1]
y_test = mitbih_test_df.iloc[:, -1]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

# Umwandlung von DataFrames in NumPy-Arrays und Hinzufügen einer Feature-Dimension
X_train = np.expand_dims(X_train.values, axis=-1)
X_test = np.expand_dims(X_test.values, axis=-1)

print("Shape von X_train:", X_train.shape)  # Sollte (Samples, Timesteps, Features) sein
print("Shape von X_test:", X_test.shape)

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Dense, Dropout, Flatten, BatchNormalization, Activation, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Residual Block
def residual_block(x, filters, kernel_size=3):
    # Shortcut-Verbindung
    shortcut = x

    if x.shape[-1] != filters:
        shortcut = Conv1D(filters=filters, kernel_size=1, padding='same')(shortcut)
    
    # Erste Conv1D-Schicht
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Zweite Conv1D-Schicht
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)

    # Hinzufügen der Shortcut-Verbindung
    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

# Anzahl der Timesteps aus den Daten bestimmen
timesteps = X_train.shape[1]

# Modellinput
inputs = Input(shape=(timesteps, 1), name="Input")

# 1D-CNN-Schicht mit den besten Hyperparametern
x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inputs)
x = MaxPooling1D(pool_size=2)(x)
x = BatchNormalization()(x)

# Residual Block 1
x = residual_block(x, filters=64)

# Residual Block 2
x = residual_block(x, filters=128)

# Residual Block 3
x = residual_block(x, filters=256)

# Flatten-Schicht
x = Flatten()(x)

# Dense-Schichten mit den besten Dropout-Werten
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)  # Bestes Dropout für Layer 1

x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)  # Bestes Dropout für Layer 2

# Output-Schicht für Mehrklassenklassifikation
output_layer = Dense(5, activation='softmax')(x)

# Modell erstellen
model = Model(inputs=inputs, outputs=output_layer)

# Modell kompilieren mit der besten Lernrate und Focal Loss
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss=loss_function,
              metrics=['accuracy'])

# Early Stopping Callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Labels in One-Hot-Encoding umwandeln
y_train_onehot = to_categorical(y_train, num_classes=5)
y_test_onehot = to_categorical(y_test, num_classes=5)

# Modell trainieren
training_history = model.fit(
    X_train,
    y_train_onehot,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping]
)

# Modell evaluieren
evaluation = model.evaluate(X_test, y_test_onehot)
print(f"Test loss: {evaluation[0]}, Test accuracy: {evaluation[1]}")

# Vorhersage

from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

report = classification_report(y_test, y_pred_classes)
print(report)

Klassengewichte: {0: 0.2416255002069822, 1: 7.877013045434098, 2: 3.025328265376641, 3: 27.317628705148206, 4: 2.7228424817291246}
(87553, 187) (87553,)
(21891, 187) (21891,)
Shape von X_train: (87553, 187, 1)
Shape von X_test: (21891, 187, 1)
Epoch 1/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 314s 135ms/step - accuracy: 0.5723 - loss: 0.9889 - val_accuracy: 0.7414 - val_loss: 0.4278
Epoch 2/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 284s 130ms/step - accuracy: 0.7805 - loss: 0.4875 - val_accuracy: 0.8622 - val_loss: 0.3176
Epoch 3/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 313s 143ms/step - accuracy: 0.8343 - loss: 0.3598 - val_accuracy: 0.9138 - val_loss: 0.2626
Epoch 4/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 332s 152ms/step - accuracy: 0.8542 - loss: 0.3342 - val_accuracy: 0.9245 - val_loss: 0.2930
Epoch 5/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 307s 140ms/step - accuracy: 0.8751 - loss: 0.2619 - val_accuracy: 0.9481 - val_loss: 0.2017
Epoch 6/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 280s 128ms/step - accuracy: 0.8942 - loss: 0

KeyboardInterrupt: 

## **Manual Feature Selection**

In [3]:
# Renaming the columns
new_column_names = range(1, len(mitbih_train_df.columns) + 1)
mitbih_train_df.columns = new_column_names

new_column_names = range(1, len(mitbih_test_df.columns) + 1)
mitbih_test_df.columns = new_column_names

# Transforming the Target Variable to Integer
mitbih_train_df[188]=mitbih_train_df[188].astype(int)

# Transforming the Target Variable to Integer
mitbih_test_df[188]=mitbih_test_df[188].astype(int)

print(mitbih_train_df[188].value_counts())

print(mitbih_test_df.shape)
print(mitbih_train_df.shape)

#Shuffle the rows of the DFs
mitbih_test_df = mitbih_test_df.sample(frac=1).reset_index(drop=True)
mitbih_train_df = mitbih_train_df.sample(frac=1).reset_index(drop=True)

mitbih_train_df.head()

188
0    72470
4     6431
2     5788
1     2223
3      641
Name: count, dtype: int64
(21891, 188)
(87553, 188)


,1,2,3,4,5,6,7,8,9,10,...,179,180,181,182,183,184,185,186,187,188
0,0.943598,0.911585,0.373476,0.000000,0.193598,0.318598,0.295732,0.282012,0.282012,0.282012,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.994681,0.845745,0.297872,0.023936,0.047872,0.061170,0.026596,0.015957,0.021277,0.007979,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1.000000,0.850394,0.230971,0.005249,0.160105,0.275591,0.267717,0.272966,0.317585,0.265092,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1.000000,0.754717,0.132075,0.015094,0.113208,0.128302,0.086792,0.147170,0.200000,0.162264,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.990548,0.930057,0.703214,0.506616,0.245747,0.085066,0.009452,0.026465,0.037807,0.094518,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [13]:
X_train = mitbih_train_df.iloc[:, 3:150]    ###

y_train = mitbih_train_df.iloc[:, -1]

X_test = mitbih_test_df.iloc[:, 3:150]    ###
y_test = mitbih_test_df.iloc[:, -1]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(87553, 147) (87553,)
(21891, 147) (21891,)


In [14]:
from tensorflow.keras.utils import to_categorical

# One-Hot-Encoding der Zielvariablen für Mehrklassenklassifikation
y_train = to_categorical(y_train, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)

In [15]:
# Using manual feature selection with my best Model so far: 

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Dense, Dropout, Flatten, BatchNormalization, Activation, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Residual Block 
def residual_block(x, filters, kernel_size=3):
    # Shortcut Connection
    shortcut = x

    if x.shape[-1] != filters:
        shortcut = Conv1D(filters=filters, kernel_size=1, padding='same')(shortcut)
    
    # First Conv1D Layer
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Second Conv1D Layer
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)

    # Adding the shortcut to the output
    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

# Modell Input
inputs = Input(shape=(147, 1), name="Input")

# 1D-CNN Layer with the best hyperparameters
x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inputs)
x = MaxPooling1D(pool_size=2)(x)
x = BatchNormalization()(x)

# Residual Block 1
x = residual_block(x, filters=64)

# Residual Block 2
x = residual_block(x, filters=128)

# Residual Block 3
x = residual_block(x, filters=256)

# Flatten Layer
x = Flatten()(x)

# Dense Layers with best Drop-Out-Values
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)  # Best Dropout für Layer 1

x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)  # Best Dropout für Layer 2

# Output Layer für Mehrklassen-Klassifikation
output_layer = Dense(5, activation='softmax')(x)

# Modell erstellen
model = Model(inputs=inputs, outputs=output_layer)

# Modell kompilieren mit der besten Lernrate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Early Stopping Callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Modell trainieren
training_history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, callbacks=[early_stopping])

# Modell evaluieren
evaluation = model.evaluate(X_test, y_test)
print(f"Test loss: {evaluation[0]}, Test accuracy: {evaluation[1]}")

Epoch 1/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 63s 28ms/step - accuracy: 0.8854 - loss: 0.4092 - val_accuracy: 0.9652 - val_loss: 0.1316
Epoch 2/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 72s 33ms/step - accuracy: 0.9632 - loss: 0.1399 - val_accuracy: 0.9753 - val_loss: 0.0898
Epoch 3/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 72s 33ms/step - accuracy: 0.9726 - loss: 0.1035 - val_accuracy: 0.9798 - val_loss: 0.0730
Epoch 4/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 69s 31ms/step - accuracy: 0.9781 - loss: 0.0806 - val_accuracy: 0.9792 - val_loss: 0.0749
Epoch 5/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 72s 33ms/step - accuracy: 0.9804 - loss: 0.0704 - val_accuracy: 0.9793 - val_loss: 0.0792
Epoch 6/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 71s 32ms/step - accuracy: 0.9831 - loss: 0.0583 - val_accuracy: 0.9821 - val_loss: 0.0723
Epoch 7/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 65s 30ms/step - accuracy: 0.9853 - loss: 0.0509 - val_accuracy: 0.9839 - val_loss: 0.0630
Epoch 8/50
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 66s 30ms/step - accuracy: 0.9868 -

In [9]:
# Prediction: 

from sklearn.metrics import classification_report
import numpy as np

y_pred = model.predict(X_test)

y_pred_classes = np.argmax(y_pred, axis=1)

y_true_classes = np.argmax(y_test, axis=1)

report = classification_report(y_true_classes, y_pred_classes)

print(report)

685/685 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     18117
           1       0.91      0.77      0.83       556
           2       0.97      0.95      0.96      1448
           3       0.83      0.79      0.81       162
           4       0.99      0.99      0.99      1608

    accuracy                           0.98     21891
   macro avg       0.94      0.90      0.92     21891
weighted avg       0.98      0.98      0.98     21891



## **What else to try:** 
- Applying a binary classification (Pathological vs. Physiologigal) on this 5-Cat-Problem
- Other combinations of feature extraction. (eventually delete rows that have Data exept of 0 behind a certain Data Point (e.g. from column 150 on) and see what it does to class distribution and then put it into practice)
- Other Over- Undersampling methods (including Data Augmentation, Adasyn)
- Further Reduction of Overfitting in Combination of CNN/RNN with L2-Regularization
- Completely different CNN Structure - eventualle 2-D-CNN (after further Preprocessing)
- Run Models with optimization for Recall
- Most importantly: Preprocessing to center around QRS-Complex - to make Interpretability logically accessible!
- *SHAP* /LIME after further Preprocessing 

### RNN Attempt: 

In [31]:
# Verwendung der vorverarbeiteten DataFrames
X_train = mitbih_train_df.iloc[:, :-1]  # Alle Spalten außer der letzten (Label)
y_train = mitbih_train_df.iloc[:, -1]   # Letzte Spalte (Label)

X_test = mitbih_test_df.iloc[:, :-1]
y_test = mitbih_test_df.iloc[:, -1]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

from tensorflow.keras.utils import to_categorical

# One-Hot-Encoding der Zielvariablen für Mehrklassenklassifikation
y_train = to_categorical(y_train, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)

# Umwandlung von DataFrames in NumPy-Arrays und Hinzufügen einer Feature-Dimension
X_train = X_train.values
X_test = X_test.values

# Reshape für LSTM (Samples, Timesteps, Features)
# Da Ihre Daten bereits in der Form (Samples, 187), müssen wir nur die Feature-Dimension hinzufügen
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

print("Shape von X_train:", X_train.shape)  # Sollte (Samples, Timesteps, Features) sein
print("Shape von X_test:", X_test.shape)

(87553, 187) (87553,)
(21891, 187) (21891,)
Shape von X_train: (87553, 187, 1)
Shape von X_test: (21891, 187, 1)


In [32]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Anzahl der Timesteps aus den Daten bestimmen
timesteps = X_train.shape[1]
input_dim = X_train.shape[2]  # Sollte 1 sein

# Modellinput
inputs = Input(shape=(timesteps, input_dim), name="Input")

# LSTM-Schichten
x = LSTM(128, return_sequences=True)(inputs)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = LSTM(64)(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

# Dense-Schichten
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)  

x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)

# Output-Schicht für Mehrklassenklassifikation
output_layer = Dense(5, activation='softmax')(x)

# Modell erstellen
model = Model(inputs=inputs, outputs=output_layer)

# Modellzusammenfassung anzeigen
model.summary()

# Modell kompilieren
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Early Stopping Callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Modell trainieren
training_history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=64,  # Möglicherweise Batch-Größe erhöhen
    callbacks=[early_stopping]
)

# Modell evaluieren
evaluation = model.evaluate(X_test, y_test)
print(f"Test loss: {evaluation[0]}, Test accuracy: {evaluation[1]}")

# Vorhersage
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

report = classification_report(y_true_classes, y_pred_classes)
print(report)

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input (InputLayer)              │ (None, 187, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 187, 128)       │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_65          │ (None, 187, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 187, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_66          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 133,637 (522.02 KB)

 Trainable params: 133,253 (520.52 KB)

 Non-trainable params: 384 (1.50 KB)

Epoch 1/50
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 232s 210ms/step - accuracy: 0.6622 - loss: 1.0786 - val_accuracy: 0.8261 - val_loss: 0.7253
Epoch 2/50
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 236s 216ms/step - accuracy: 0.8230 - loss: 0.7401 - val_accuracy: 0.8260 - val_loss: 0.6922
Epoch 3/50
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 242s 221ms/step - accuracy: 0.8239 - loss: 0.6909 - val_accuracy: 0.8259 - val_loss: 0.7313
Epoch 4/50
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 265s 242ms/step - accuracy: 0.8248 - loss: 0.6313 - val_accuracy: 0.8265 - val_loss: 0.6112
Epoch 5/50
 258/1095 ━━━━━━━━━━━━━━━━━━━━ 2:46 199ms/step - accuracy: 0.8319 - loss: 0.5941

KeyboardInterrupt: 

### **Transformer:**

In [22]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, LayerNormalization, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [3]:
X_train = mitbih_train_df.iloc[:, :-1]  
y_train = mitbih_train_df.iloc[:, -1] 

X_test = mitbih_test_df.iloc[:, :-1]
y_test = mitbih_test_df.iloc[:, -1]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

from tensorflow.keras.utils import to_categorical

# One-Hot-Encoding der Zielvariablen für Mehrklassenklassifikation
y_train = to_categorical(y_train, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)

# Umwandlung von DataFrames in NumPy-Arrays und Hinzufügen einer Feature-Dimension
X_train = np.expand_dims(X_train.values, axis=-1)
X_test = np.expand_dims(X_test.values, axis=-1)

print("Shape von X_train:", X_train.shape)  # Sollte (Samples, Timesteps, Features) sein
print("Shape von X_test:", X_test.shape)

(87553, 187) (87553,)
(21891, 187) (21891,)
Shape von X_train: (87553, 187, 1)
Shape von X_test: (21891, 187, 1)


In [25]:
# Positions-Encodings Funktion
def positional_encoding(maxlen, embed_dim):
    positions = np.arange(maxlen)[:, np.newaxis]
    dims = np.arange(embed_dim)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (dims // 2)) / np.float32(embed_dim))
    angle_rads = positions * angle_rates
    # Anwenden von sin auf die geraden Indizes und cos auf die ungeraden Indizes
    sines = np.sin(angle_rads[:, 0::2])
    cosines = np.cos(angle_rads[:, 1::2])
    pos_encoding = np.concatenate([sines, cosines], axis=-1)
    return tf.cast(pos_encoding, dtype=tf.float32)

In [26]:
# Transformer Encoder Block
class TransformerEncoderBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerEncoderBlock, self).__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [
                tf.keras.layers.Dense(ff_dim, activation='relu'),
                tf.keras.layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)
    
    def call(self, inputs, training=None):
        attn_output = self.att(inputs, inputs, training=training)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        
        ffn_output = self.ffn(out1, training=training)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


In [27]:
# Parameter
embed_dim = 64  # Erhöht von 32 auf 64
num_heads = 8   # Erhöht von 4 auf 8
ff_dim = 256    # Erhöht von 64 auf 256
num_transformer_blocks = 4  # Erhöht von 2 auf 4, um die Tiefe zu steigern

# Anzahl der Timesteps aus den Daten bestimmen
timesteps = X_train.shape[1]
input_dim = 1

# Modellinput
inputs = Input(shape=(timesteps, input_dim))

# Positions-Encodings hinzufügen
positions = positional_encoding(timesteps, embed_dim)
x = Dense(embed_dim)(inputs)
x = x + positions  # Positionsinformationen hinzufügen

# Mehrere Transformer Encoder Blöcke
for _ in range(num_transformer_blocks):
    transformer_block = TransformerEncoderBlock(embed_dim, num_heads, ff_dim)
    x = transformer_block(x)

# Global Average Pooling
x = tf.keras.layers.GlobalAveragePooling1D()(x)

# Klassifikationskopf mit Batch-Normalisierung und reduzierten Dropout-Raten
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

# Output-Schicht
outputs = Dense(5, activation='softmax')(x)

# Modell erstellen
model = Model(inputs=inputs, outputs=outputs)

# Modellzusammenfassung anzeigen
model.summary()

# Optimierer - leicht erhöhte Lernrate
optimizer = Adam(learning_rate=0.001)

# Lernraten-Reduzierung bei Plateau
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1, min_lr=1e-5)

# Early Stopping Callback
early_stopping = EarlyStopping(monitor="val_accuracy", patience=5, restore_best_weights=True)

# Modell kompilieren
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

# Modell trainieren
training_history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=64,
    callbacks=[early_stopping, reduce_lr]
)

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 187, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 187, 64)        │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ add_1 (Add)                     │ (None, 187, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder_block_4     │ (None, 187, 64)        │       166,016 │
│ (TransformerEncoderBlock)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder_block_5     │ (None, 187, 64)        │       166,016 │
│ (TransformerEncoderBlock)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder_block_6     │ (None, 187, 64)        │       166,016 │
│ (TransformerEncoderBlock)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder_block_7     │ (None, 187, 64)        │       166,016 │
│ (TransformerEncoderBlock)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ (None, 64)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 256)            │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 715,909 (2.73 MB)

 Trainable params: 715,141 (2.73 MB)

 Non-trainable params: 768 (3.00 KB)

Epoch 1/50
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 506s 456ms/step - accuracy: 0.7324 - loss: 0.8547 - val_accuracy: 0.9169 - val_loss: 0.3237 - learning_rate: 0.0010
Epoch 2/50
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 514s 469ms/step - accuracy: 0.9410 - loss: 0.2320 - val_accuracy: 0.6597 - val_loss: 1.2970 - learning_rate: 0.0010
Epoch 3/50
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 528s 482ms/step - accuracy: 0.9500 - loss: 0.1882 - val_accuracy: 0.9516 - val_loss: 0.2308 - learning_rate: 0.0010
Epoch 4/50
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 513s 469ms/step - accuracy: 0.9540 - loss: 0.1727 - val_accuracy: 0.9380 - val_loss: 0.3122 - learning_rate: 0.0010
Epoch 5/50
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 552s 504ms/step - accuracy: 0.9580 - loss: 0.1577 - val_accuracy: 0.1251 - val_loss: 3.0790 - learning_rate: 0.0010
Epoch 6/50
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 0s 448ms/step - accuracy: 0.9634 - loss: 0.1418
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 546s 498ms/st

KeyboardInterrupt: 

In [ ]:
# Modell evaluieren
evaluation = model.evaluate(X_test, y_test)
print(f"Test loss: {evaluation[0]}, Test accuracy: {evaluation[1]}")

# Vorhersage
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

report = classification_report(y_true_classes, y_pred_classes)
print(report)

685/685 ━━━━━━━━━━━━━━━━━━━━ 26s 38ms/step - accuracy: 0.9793 - loss: 0.0809
Test loss: 0.0761103555560112, Test accuracy: 0.9804942607879639
685/685 ━━━━━━━━━━━━━━━━━━━━ 29s 41ms/step
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     18117
           1       0.86      0.72      0.79       556
           2       0.96      0.94      0.95      1448
           3       0.83      0.70      0.76       162
           4       0.99      0.98      0.99      1608

    accuracy                           0.98     21891
   macro avg       0.93      0.87      0.89     21891
weighted avg       0.98      0.98      0.98     21891



2nd Attempt Transformer: 

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, LayerNormalization, Embedding, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

timesteps = 187
input_dim = 1

# Transformer Encoder Block
class TransformerEncoderBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerEncoderBlock, self).__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [
                tf.keras.layers.Dense(ff_dim, activation='relu'),
                tf.keras.layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)
    
    def call(self, inputs, training=None):
        attn_output = self.att(inputs, inputs, training=training)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        
        ffn_output = self.ffn(out1, training=training)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Parameter für den hybriden Ansatz
embed_dim = 64    # Dimension des embeddings nach CNN
num_heads = 8     # Anzahl der Köpfe in Multi-Head Attention
ff_dim = 256      # Feed-Forward-Dimension
num_transformer_blocks = 2  
num_filters = 32  # Anzahl der Filter für die Convolutional-Schicht
kernel_size = 3   # Kernel-Größe für die Convolution

# Modellinput
inputs = Input(shape=(timesteps, input_dim))

# Convolutional & Pooling-Schichten
conv_out = Conv1D(filters=num_filters, kernel_size=kernel_size, activation='relu', padding='same')(inputs)
conv_out = MaxPooling1D(pool_size=2)(conv_out)
conv_out = BatchNormalization()(conv_out)

# Dense-Schicht, um Convolution-Ausgang auf Embed-Dimension zu bringen
x = Dense(embed_dim)(conv_out)

# Lernbare Positions-Embeddings
reduced_timesteps = x.shape[1]  # Neue Zeitschrittlänge nach Pooling
position_input = tf.range(start=0, limit=reduced_timesteps, delta=1)
position_embedding_layer = Embedding(input_dim=reduced_timesteps, output_dim=embed_dim)
position_embeddings = position_embedding_layer(position_input)
position_embeddings = tf.expand_dims(position_embeddings, axis=0)

# Positionsinformationen hinzufügen
x = x + position_embeddings

# Transformer Encoder Blöcke
for _ in range(num_transformer_blocks):
    transformer_block = TransformerEncoderBlock(embed_dim, num_heads, ff_dim)
    x = transformer_block(x)

# Global Average Pooling
x = tf.keras.layers.GlobalAveragePooling1D()(x)

# Klassifikationskopf
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

# Output-Schicht
outputs = Dense(5, activation='softmax')(x)

# Modell erstellen
model = Model(inputs=inputs, outputs=outputs)

# Modellzusammenfassung
model.summary()

# Optimierer
optimizer = Adam(learning_rate=0.0005)

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, verbose=1, min_lr=1e-5)
early_stopping = EarlyStopping(monitor="val_accuracy", patience=5, restore_best_weights=True)

# Modell kompilieren
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

# Modell trainieren
training_history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=64,
    callbacks=[early_stopping, reduce_lr]
)

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 187, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 187, 32)        │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 93, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 93, 32)         │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 93, 64)         │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ add_1 (Add)                     │ (None, 93, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder_block_4     │ (None, 93, 64)         │       166,016 │
│ (TransformerEncoderBlock)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder_block_5     │ (None, 93, 64)         │       166,016 │
│ (TransformerEncoderBlock)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ (None, 64)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 352,069 (1.34 MB)

 Trainable params: 351,621 (1.34 MB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/50
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 101s 88ms/step - accuracy: 0.7264 - loss: 0.8920 - val_accuracy: 0.9149 - val_loss: 0.2982 - learning_rate: 5.0000e-04
Epoch 2/50
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 118s 108ms/step - accuracy: 0.9310 - loss: 0.2600 - val_accuracy: 0.9052 - val_loss: 0.3028 - learning_rate: 5.0000e-04
Epoch 3/50
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 106s 97ms/step - accuracy: 0.9453 - loss: 0.1946 - val_accuracy: 0.9531 - val_loss: 0.1643 - learning_rate: 5.0000e-04
Epoch 4/50
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 117s 107ms/step - accuracy: 0.9493 - loss: 0.1808 - val_accuracy: 0.9596 - val_loss: 0.1506 - learning_rate: 5.0000e-04
Epoch 5/50
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 107s 98ms/step - accuracy: 0.9555 - loss: 0.1564 - val_accuracy: 0.9589 - val_loss: 0.1491 - learning_rate: 5.0000e-04
Epoch 6/50
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 116s 106ms/step - accuracy: 0.9582 - loss: 0.1432 - val_accuracy: 0.9363 - val_loss: 0.2347 - learning_rate: 5.0000e-04
Epoch 7/50
1095/1095 ━━━━━━━━━━

In [ ]:
# Modell evaluieren
evaluation = model.evaluate(X_test, y_test)
print(f"Test loss: {evaluation[0]}, Test accuracy: {evaluation[1]}")

# Vorhersage
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

report = classification_report(y_true_classes, y_pred_classes)
print(report)